In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import random
import sympy
import scipy.stats as st
import scipy.special
import pandas
import csv
import math
import multiprocessing as mp
import datetime
from decimal import *
from mpl_toolkits.mplot3d import Axes3D
from google.colab import drive

## **Google Drive**

In [2]:
# connect to google drive to save the csv file
drive.mount('drive')
os.chdir("drive/My Drive/Yen-Ting Lin/Code")

Mounted at drive


## **Back-off Calculation**

### **Retransmission at same RAP**

In [3]:
# alpha with i
def find_kmin(T_RAREP, T_RAR, W_RAR, W_BO, i):
  return np.ceil((i - 1) + ((1 - (T_RAR + W_RAR + W_BO)) / T_RAREP))
  # return np.ceil((i - 1) - (((T_RAR + W_RAR + W_BO) - 1) / T_RAREP)) # same equation

def find_kmax(T_RAREP, T_RAR, W_RAR, W_BO, i):
  return np.floor(i - ((T_RAR + W_RAR + 1) / T_RAREP))

def find_alpha(T_RAREP, T_RAR, W_RAR, W_BO, i, k, kmin, kmax):
  if kmin <= k <= (i - ((T_RAR + W_RAR + W_BO) / T_RAREP)):
    return (((k - 1) * T_RAREP) + T_RAR + W_RAR + W_BO - ((i - 2) * T_RAREP)) / W_BO
  elif (i - ((T_RAR + W_RAR + W_BO) / T_RAREP)) < k < ((i - 1) - ((T_RAR + W_RAR) / T_RAREP)):
    return ((T_RAREP) / W_BO)
  elif ((i - 1) - ((T_RAR + W_RAR) / T_RAREP)) <= k <= kmax:
    return (((i - 1) * T_RAREP) - (((k - 1) * T_RAREP) + T_RAR + W_RAR)) / W_BO
  else:
    return 0

### **Retransmission of fallback RAP**

In [4]:
def find_jmin(T_RAREP_4sra, T_RAREP_2sra, T_RAR, W_RAR, i):
  return np.ceil((((i - 2) * T_RAREP_4sra) + T_RAR + W_RAR - 1) / T_RAREP_2sra) + 1

def find_jmax(T_RAREP_4sra, T_RAREP_2sra, T_RAR, W_RAR, i):
  return np.floor((((i - 1) * T_RAREP_4sra) + T_RAR + W_RAR - 2) / T_RAREP_2sra) + 1

## **Main Function**

In [34]:
def main():
  # initail number of fallback UE
  M_FB = [0.0 for i in range(0, M - 1)]

  Ms_4SRA_list = [0.0 for i in range(0, len(M_4SRA))]
  Ps_4SRA_list = [0.0 for i in range(0, len(M_4SRA))]
  Da_4SRA_list = [0.0 for i in range(0, len(M_4SRA))]

  Ms_2SRA_list = [0.0 for i in range(0, len(M_2SRA))]
  Ps_2SRA_list = [0.0 for i in range(0, len(M_2SRA))]
  Da_2SRA_list = [0.0 for i in range(0, len(M_2SRA))]

  Ms_FB_list = [0.0 for i in range(0, len(M_2SRA))]
  Ps_FB_list = [0.0 for i in range(0, len(M_2SRA))]
  Da_FB_list = [0.0 for i in range(0, len(M_2SRA))]

  Ms_T_list = [0.0 for i in range(0, M - 1)]
  Ps_T_list = [0.0 for i in range(0, M - 1)]
  Da_T_list = [0.0 for i in range(0, M - 1)]

  for ue in range(0, M - 1):
    # initial the MiSn and MiFn: Mi[slot][num of trans + 1]
    # the last time of retrans needed to be ignored
    MiCn_4SRA = [[0.0] * int((retrans_4sra + 1) + 1) for _ in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) + 1)]
    MiSn_4SRA = [[0.0] * int((retrans_4sra + 1) + 1) for _ in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) + 1)]
    MiFn_4SRA = [[0.0] * int((retrans_4sra + 1) + 1) for _ in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) + 1)]
    T_4SRA = [0.0 for i in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) + 1)]

    MiCn_2SRA = [[0.0] * int((retrans_2sra + 1) + 1) for _ in range(0, maxslot(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, retrans_2sra) + 1)]
    MiSn_2SRA = [[0.0] * int((retrans_2sra + 1) + 1) for _ in range(0, maxslot(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, retrans_2sra) + 1)]
    MiFn_2SRA = [[0.0] * int((retrans_2sra + 1) + 1) for _ in range(0, maxslot(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, retrans_2sra) + 1)]
    T_2SRA = [0.0 for i in range(0, maxslot(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, retrans_2sra) + 1)]

    MiCn_FB = [[0.0] * int((retrans_4sra + 1) + 1) for _ in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) + 1)]
    MiSn_FB = [[0.0] * int((retrans_4sra + 1) + 1) for _ in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) + 1)]
    MiFn_FB = [[0.0] * int((retrans_4sra + 1) + 1) for _ in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) + 1)]
    T_FB = [0.0 for i in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) + 1)]

    # num of total UE in slot 0
    MiFn_2SRA[0][0] = M_2SRA[ue]
    # print(np.matrix(MiFn))

    # 2SRA procedure sure be finished before 4SRA procedure
    for i in range(0, maxslot(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, retrans_2sra)):
      # print("i = ", i, "\n")
      kmin = find_kmin(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, i)
      kmax = find_kmax(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, i)
      # print("kmin = ", kmin, "kmax = ", kmax, "\n")
      m_2SRA = MiFn_2SRA[i][0] + MiFn_2SRA[i][1] + MiFn_2SRA[i][2] + MiFn_2SRA[i][3] # Mi

      for n in range(0, int(retrans_2sra + 1)):
        # print("n = ", n, "\n")
        k_list_2SRA = []
        if MiFn_2SRA[i][n] != 0:
          for k in range(int(kmin), int(kmax + 1)):
            # print("k = ", k, "\n")
            if N_2SRA[0] == 0:
              E_M = 0
            else:
              E_M = np.exp(-(m_2SRA / N_2SRA[0]))
            alpha = find_alpha(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, i, k, kmin, kmax)
            k_list_2SRA.append(alpha * (MiFn_2SRA[i][n] * (1.0 - E_M)))
            # print("alpha = ", alpha, "\n")
          # process the data to put it into array like type 1
          for itr_k in range(0, len(k_list_2SRA)):
            MiFn_2SRA[i + itr_k + 1][n + 1] += k_list_2SRA[(len(k_list_2SRA) - 1) - itr_k]
            # print("MiCn = ", MiFn[i][n] * (1.0 - E_M), "\n")
            # print(np.matrix(MiFn))
      for n in range(0, int(retrans_2sra + 1)):
        if N_2SRA[0] == 0:
          E_M = 0
        else:
          E_M = np.exp(-(m_2SRA / N_2SRA[0]))
        MiSn_2SRA[i][n] = MiFn_2SRA[i][n] * E_M
        MiCn_2SRA[i][n] = MiFn_2SRA[i][n] * (1.0 - E_M)
      # define delay
      T_2SRA[i] = (i + 1) * T_RAREP_2sra + T_RAR_2sra + W_RAR_2sra

    # num of total UE in slot 0
    MiFn_4SRA[0][0] = M_4SRA[ue]
    # sum all the failed 2SRA UE to become total number of M_FB
    if T_RAREP_4sra < T_RAREP_2sra:
      for i in range(0, maxslot(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, retrans_2sra)):
        # if (MiCn_2SRA[i][3]) != 0: -> cannot use this condition, it will make error
        M_FB[ue] += MiCn_2SRA[i][3]
        # print(M_FB[ue], MiCn_2SRA[i])
        # jmin and jmax must minus 2 here to fix the problem caused by starting slot i = 0
        jmin = find_jmin(T_RAREP_4sra, T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, i) - 2
        jmax = find_jmax(T_RAREP_4sra, T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, i) - 2
        # print(i - 1, jmax, jmin)
        if jmax == jmin:
          MiFn_FB[i - 1][0] = MiCn_2SRA[int(jmax)][3]
        # print(MiFn_FB[i])
    if T_RAREP_4sra == T_RAREP_2sra:
      for i in range(0, maxslot(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, retrans_2sra)):
        if (MiCn_2SRA[i][3]) != 0:
          M_FB[ue] += MiCn_2SRA[i][3]
          # print(M_FB[ue], MiCn_2SRA[i])
          jmax = find_jmax(T_RAREP_4sra, T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, i) - 2
          # print(i, jmax)
          MiFn_FB[i - 1][0] = MiCn_2SRA[int(jmax)][3]
          # print(MiFn_FB[i])
    if T_RAREP_4sra > T_RAREP_2sra:
      # max value of i iteration could be 4sra or 2sra, they will still have the same result
      for i in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra)):
        if (MiCn_2SRA[i][3]) != 0:
          M_FB[ue] += MiCn_2SRA[i][3]
          # print(MiCn_2SRA[i])
          # jmin and jmax must minus 2 here to fix the problem caused by starting slot i = 0
          jmin = find_jmin(T_RAREP_4sra, T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, i) - 2
          jmax = find_jmax(T_RAREP_4sra, T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, i) - 2
          # print(i, jmin, jmax)
          for j in range(int(jmin), int(jmax + 1)):
            MiFn_FB[i - 1][0] += MiCn_2SRA[j][3]
          # print(MiFn_FB[i])
          # print("---------------------------------------------")
    # print(np.matrix(MiCn_2SRA))
    # print(np.matrix(MiFn_FB))

    # 4SRA procedure
    for i in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra)):
      # print("i = ", i, "\n")
      kmin = find_kmin(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, i)
      kmax = find_kmax(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, i)
      # print("kmin = ", kmin, "kmax = ", kmax, "\n")
      m_4SRA = MiFn_4SRA[i][0] + MiFn_4SRA[i][1] + MiFn_4SRA[i][2] + MiFn_4SRA[i][3] # Mi
      m_FB = MiFn_FB[i][0] + MiFn_FB[i][1] + MiFn_FB[i][2] + MiFn_FB[i][3] # Mi

      for n in range(0, int(retrans_4sra + 1)):
        # print("n = ", n, "\n")
        k_list_4SRA = []
        k_list_FB = []
        if (MiFn_4SRA[i][n] + MiFn_FB[i][n]) != 0:
          for k in range(int(kmin), int(kmax + 1)):
            # print("k = ", k, "\n")
            if N_4SRA[0] == 0:
              E_M = 0
            else:
              E_M = np.exp(-((m_4SRA + m_FB) / N_4SRA[0]))
            alpha = find_alpha(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, i, k, kmin, kmax)
            k_list_4SRA.append(alpha * (MiFn_4SRA[i][n] * (1.0 - E_M)))
            k_list_FB.append(alpha * (MiFn_FB[i][n] * (1.0 - E_M)))
            # print("alpha = ", alpha, "\n")
          # process the data to put it into array like type 1
          for itr_k in range(0, len(k_list_4SRA)):
            # itr_k must plus 1 here to fix the problem caused by starting from slot i = 0
            MiFn_4SRA[i + itr_k + 1][n + 1] += k_list_4SRA[(len(k_list_4SRA) - 1) - itr_k]
            MiFn_FB[i + itr_k + 1][n + 1] += k_list_FB[(len(k_list_FB) - 1) - itr_k]
            # print("MiCn = ", MiFn[i][n] * (1.0 - E_M), "\n")
            # print(np.matrix(MiFn))
      for n in range(0, int(retrans_4sra + 1)):
        if N_4SRA[0] == 0:
          E_M = 0
        else:
          E_M = np.exp(-((m_4SRA + m_FB) / N_4SRA[0]))
        MiSn_4SRA[i][n] = MiFn_4SRA[i][n] * E_M
        MiCn_4SRA[i][n] = MiFn_4SRA[i][n] * (1.0 - E_M)
        MiSn_FB[i][n] = MiFn_FB[i][n] * E_M
        MiCn_FB[i][n] = MiFn_FB[i][n] * (1.0 - E_M)
      # define delay
      T_4SRA[i] = (i + 1) * T_RAREP_4sra + T_RAR_4sra + W_RAR_4sra
      T_FB[i] = (i + 1) * T_RAREP_4sra + T_RAR_4sra + W_RAR_4sra

    # Ms, Ps, Da for 2SRA
    for i in range(0, maxslot(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, retrans_2sra)):
      for n in range(0, int(retrans_2sra + 1)):
        # sum the number of succeed UE for every transmission in each slot
        Ms_2SRA_list[ue] += MiSn_2SRA[i][n]
        Da_2SRA_list[ue] += MiSn_2SRA[i][n] * float(T_2SRA[i])

    # Ms, Ps, Da for 4SRA
    for i in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra)):
      for n in range(0, int(retrans_4sra + 1)):
        # sum the number of succeed UE for every transmission in each slot
        Ms_4SRA_list[ue] += MiSn_4SRA[i][n]
        Ms_FB_list[ue] += MiSn_FB[i][n]
        Da_4SRA_list[ue] += MiSn_4SRA[i][n] * float(T_4SRA[i])
        Da_FB_list[ue] += MiSn_FB[i][n] * float(T_FB[i])

    # total number of succeed UE when there have k UE in the system
    Ms_4SRA_list[ue] = round(Ms_4SRA_list[ue] * 1000000) / 1000000
    Ms_2SRA_list[ue] = round(Ms_2SRA_list[ue] * 1000000) / 1000000
    Ms_FB_list[ue] = round(Ms_FB_list[ue] * 1000000) / 1000000
    Ms_T_list[ue] = round((Ms_4SRA_list[ue] + Ms_2SRA_list[ue] + Ms_FB_list[ue]) * 1000000) / 1000000

    if M_4SRA[ue] != 0:
      Ps_4SRA_list[ue] = round((Ms_4SRA_list[ue] / M_4SRA[ue]) * 1000000) / 1000000
    else:
      Ps_4SRA_list[ue] = 0
    if M_2SRA[ue] != 0:
      Ps_2SRA_list[ue] = round((Ms_2SRA_list[ue] / M_2SRA[ue]) * 1000000) / 1000000
    else:
      Ps_2SRA_list[ue] = 0
    if M_FB[ue] != 0:
      Ps_FB_list[ue] = round((Ms_FB_list[ue] / M_FB[ue]) * 1000000) / 1000000
    else:
      Ps_FB_list[ue] = 0
    if M_4SRA[ue] + M_2SRA[ue] != 0:
      Ps_T_list[ue] = round((Ms_4SRA_list[ue] + Ms_2SRA_list[ue] + Ms_FB_list[ue]) / (M_4SRA[ue] + M_2SRA[ue]) * 1000000) / 1000000
    else:
      Ps_T_list[ue] = 0

    if M_4SRA[ue] != 0:
      Da_4SRA_list[ue] = float(Da_4SRA_list[ue]) / float(Ms_4SRA_list[ue])
    else:
      Da_4SRA_list[ue] = 0.0
    if M_2SRA[ue] != 0:
      Da_2SRA_list[ue] = float(Da_2SRA_list[ue]) / float(Ms_2SRA_list[ue])
    else:
      Da_2SRA_list[ue] = 0.0
    if M_FB[ue] != 0 and Ms_FB_list[ue] != 0:
      Da_FB_list[ue] = float(Da_FB_list[ue]) / float(Ms_FB_list[ue])
    else:
      Da_FB_list[ue] = 0.0
    if M_4SRA[ue] + M_2SRA[ue] != 0:
      Da_T_list[ue] = (Da_4SRA_list[ue] * float(Ms_4SRA_list[ue]) + Da_2SRA_list[ue] * float(Ms_2SRA_list[ue]) + Da_FB_list[ue] * float(Ms_FB_list[ue])) / (float(Ms_4SRA_list[ue]) + float(Ms_2SRA_list[ue]) + float(Ms_FB_list[ue]))
    else:
      Da_T_list[ue] = 0.0

    # print(len(M), len(Ms_list), len(Ps_list), len(Da_list))

  return M_4SRA, Ms_4SRA_list, Ps_4SRA_list, Da_4SRA_list, M_2SRA, Ms_2SRA_list, Ps_2SRA_list, Da_2SRA_list, M_FB, Ms_FB_list, Ps_FB_list, Da_FB_list, Ms_T_list, Ps_T_list, Da_T_list

## **Initial Parameters**

In [35]:
# number of preamble
N_4SRA = [44]
N_2SRA = [10]

# number of max retransmission
retrans_4sra = 3 # N_PTmax - 1
retrans_2sra = 3

# Time Duration (ms)
T_RAREP_4sra = 1780
T_RAR_4sra = 6.4 + 3 # ra-ResponseWindow
W_RAR_4sra = 16
T_N_CR_4sra = 32
W_BO_4sra = 4096

# Time Duration (ms)
T_RAREP_2sra = 1280
T_RAR_2sra = 6.4 + 3 # ra-ResponseWindow
W_RAR_2sra = 16
T_N_CR_2sra = 32
W_BO_2sra = 4096

def maxslot(T_RAREP, T_RAR, W_RAR, W_BO, retrans):
  Maxraslot = np.ceil((T_RAR + W_RAR + W_BO) / T_RAREP) * (retrans + 1) + 1
  return int(Maxraslot) + 100
# print(maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) - 100)
# print(maxslot(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, retrans_2sra) - 100)
'''
# total number of UE
M = 1000
# initail number of 2SRA UE
M_4SRA = []
M_2SRA = []
for i in range(1, 1000):
  M_2SRA.append(i)
  M_4SRA.append(M - i)
'''
M = 2
M_4SRA = [1]
M_2SRA = [999]


## **Test Result**

In [36]:
columns = ["M_4SRA", "N_4SRA", "Ms_4SRA", "Ps_4SRA", "Da_4SRA", "M_2SRA", "N_2SRA", "Ms_2SRA", "Ps_2SRA", "Da_2SRA", "M_FB", "Ms_FB", "Ps_FB", "Da_FB", "Ms_T", "Ps_T", "Da_T"]
M_4SRA, Ms_4SRA_list, Ps_4SRA_list, Da_4SRA_list, M_2SRA, Ms_2SRA_list, Ps_2SRA_list, Da_2SRA_list, M_FB, Ms_FB_list, Ps_FB_list, Da_FB_list, Ms_T_list, Ps_T_list, Da_T_list = main()

N_4SRA_list = [N_4SRA[0] for i in range(0, M - 1)]
N_2SRA_list = [N_2SRA[0] for i in range(0, M - 1)]

data = {"M_4SRA" : M_4SRA, "N_4SRA" : N_4SRA_list, "Ms_4SRA" : Ms_4SRA_list, "Ps_4SRA" : Ps_4SRA_list, "Da_4SRA" : Da_4SRA_list,
        "M_2SRA" : M_2SRA, "N_2SRA" : N_2SRA_list, "Ms_2SRA" : Ms_2SRA_list, "Ps_2SRA" : Ps_2SRA_list, "Da_2SRA" : Da_2SRA_list,
        "M_FB": M_FB, "Ms_FB": Ms_FB_list, "Ps_FB": Ps_FB_list, "Da_FB": Da_FB_list,
        "Ms_T" : Ms_T_list, "Ps_T" : Ps_T_list, "Da_T" : Da_T_list}

result = pandas.DataFrame(data)
# cvs_pandas = result.to_csv("ana_testresult.csv", mode='a', index=False, header=True, columns=columns)
cvs_pandas = result.to_csv("ana_testresult.csv", mode='w+', index=False, header=True, columns=columns)

## **Plotting Figures**

In [37]:
ana = pandas.read_csv('ana_testresult.csv')
# sim = pandas.read_csv('sim_testresult.csv')
ana

,M_4SRA,N_4SRA,Ms_4SRA,Ps_4SRA,Da_4SRA,M_2SRA,N_2SRA,Ms_2SRA,Ps_2SRA,Da_2SRA,M_FB,Ms_FB,Ps_FB,Da_FB,Ms_T,Ps_T,Da_T
0,1,44,0.998603,0.998603,1866.534571,999,10,5.306845,0.005312,14931.965952,993.693155,64.048304,0.064455,18890.86178,70.353752,0.070354,18350.595061
